## Welcome to the Q-SEnSE Hackathon!
### This is a warmup challenge designed to give you basic familiarity with writing quantum circuits in CUDA Quantum and Supermarq (which uses Google's quantum package Cirq).

#### The goal of this challenge is to write the simplest (yet still useful) quantum circuit: the quantum random number generator! ($25)

Your mission, should you choose to accept it, is to write a quantum random number generator (QMRG) in either CUDA Quantum or Supermarq (or both!). This QMRG will take the form of a function that takes as input an integer x and outputs a random number between 0 and x-1. 

Hint: This function should behave just like the pseudorandom python function randrange(x) but due to its use of quantum randomness is truely random not just pseudorandom!


#### Example Code:

Here is an example of the random number generator written in Qiskit (IBM's quantum package) - your code should reproduce the random bit generator in Cirq or CUDA Quantum in such a way as to display a random integer (this code produces a random n-bit string instead of an integer)!

In [2]:
###
#Generating a Qrandom number between 0 and n-1 in Qiskit
###

def Qrand(n):

    import numpy as np
    from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
    from qiskit.providers.aer import QasmSimulator
    from qiskit.visualization import plot_histogram


    # Use Aer's qasm_simulator
    simulator = QasmSimulator()

    # Create a Quantum Circuit acting on the q register
    qr = QuantumRegister(n, 'q')
    cr = ClassicalRegister(n, 'c')
    circuit = QuantumCircuit(qr, cr)


# Add a H gate on all qubits
    for i in range(0,n):
            circuit.h(i)


# Map the quantum measurement to the classical bits
    circuit.measure(qr, cr)

# compile the circuit down to low-level QASM instructions
# supported by the backend (not needed for simple circuits)
    compiled_circuit = transpile(circuit, simulator)

# Execute the circuit on the qasm simulator
    job = simulator.run(compiled_circuit, shots=1)

# Grab results from the job
    result = job.result()


# Returns counts
    counts = result.get_counts(compiled_circuit)
    print(counts)

This example run returns an output in the form {random bit string : probability of that string}

In [2]:
Qrand(4)

{'0110': 1}


In [3]:
Qrand(10)

{'0100111101': 1}


In [3]:
import collections

import cirq
from cirq.contrib.qasm_import import circuit_from_qasm
from qiskit.synthesis import generate_basic_approximations
from qiskit.transpiler.passes.synthesis import SolovayKitaev

import supermarq
from supermarq.benchmark import Benchmark
from supermarq.benchmarks.bit_code import BitCode
from supermarq.benchmarks.phase_code import PhaseCode
from supermarq.benchmarks.ghz import GHZ
from supermarq.benchmarks.hamiltonian_simulation import HamiltonianSimulation
from supermarq.benchmarks.mermin_bell import MerminBell
from supermarq.benchmarks.qaoa_fermionic_swap_proxy import QAOAFermionicSwapProxy
from supermarq.benchmarks.qaoa_vanilla_proxy import QAOAVanillaProxy
from supermarq.benchmarks.vqe_proxy import VQEProxy

In [4]:
# Utilities
def compile_to_clifford_t(circuit: cirq.Circuit) -> cirq.Circuit:
    """Compiles to the clifford + t gateset (H, S, CNOT, T)"""
    qiskit_circuit = supermarq.converters.cirq_to_qiskit(circuit)
    qiskit_circuit.remove_final_measurements()
    basis = ["h", "s", "t"]
    approx = generate_basic_approximations(basis, depth=3)

    skd = SolovayKitaev(recursion_degree=2, basic_approximations=approx)
 
    discretized = skd(qiskit_circuit)

    qasm = discretized.qasm()
 
    return circuit_from_qasm(qasm)


def results_to_counts(circuit: cirq.Circuit, results: cirq.Result) -> collections.Counter[str]:
    """Transforms cirq.Result in collection.Counter"""
    new_collections_counter: collections.Counter[str] = collections.Counter()
    keys = sorted(circuit.all_measurement_key_names())
    num_qubits_measured_per_key = [results.measurements[key].shape[1] for key in keys]
    histogram = results.multi_measurement_histogram(keys=keys)

    for old_keys in histogram:
        new_key = "".join(
                f"{old_key:>0{num_qubits_measured}b}"
                for old_key, num_qubits_measured in zip(old_keys, num_qubits_measured_per_key)
            )
        new_collections_counter[new_key] = histogram[old_keys]

    return new_collections_counter

In [5]:
class BitCode(BitCode):
    def circuit(self) -> cirq.Circuit:
        pass

In [9]:
#ask user for the top range number
def userNum():
    topNum = int(input("Enter the top range number: "))
    return topNum

#figure out how many bits the users input will be represented by
def userBits(topNum):
    binary = bin(topNum)
    bits = binary.bit_length()
    return bits

def rand(bits, topNum):
    #Creates a quantum circuit
    circuit = cirq.Circuit(cirq.depolarize(0.5).on('a'), cirq.measure('a'))
    simulator = cirq.Simulator()
    result = simulator.run(circuit, repetitions = bits)

    for i in range(topNum):
        qubit = cirq.LineQubit(i)
        circuit.append(cirq.H(qubit))

    generated_nums = []
    for measurements in result.measurements['a']:
        generated_num = int(''.join(str(bit) for bit in measurements), 2)
        generated_nums.append(generated_num)

    return generated_nums

def main():
    tNum = userNum()
    bNum = userBits(tNum)
    gNum = None

    while gNum is None or gNum > tNum -1:
        gNum = rand(bNum, tNum)

    print("Number generated: ", gNum)

    if __name__ == "__main_-":
        main()